In [ ]:
#========================================================================================================================
# sentence-bert(sbert)에 CrossEncoder 방식 테스트 예시임
# => cross-encocoder 방식은 2개의 문장(문장1, 문장2)을 입력했을때 output으로 유사도(0~1값)을 출력해줌
#
# => 입력 모델은 bert모델, distibert, roberta 등 됨(NLI, STS 학습된 모델이 성능이 좋음)
#    **단 NLI, STS 학습된 bert 모델인 경우에는 bertmodel로 만들어야 함.
#     왜냐하면 NLI는 classifiation 모델이므로, 출력값(참, 거짓, 중립)이 3개이므로, 출력은 임베딩값 출력되는 모델로 변환시켜야함
#
# => 참고 : https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/cross-encoder/cross-encoder_usage.py
#========================================================================================================================
import torch 
import os
import numpy as np
from sentence_transformers.cross_encoder import CrossEncoder
from os import sys
sys.path.append('../../')
from myutils import seed_everything, GPU_info, mlogging
import time

seed_everything(111)
device = GPU_info()
logger =  mlogging(loggername="sbertcross", logfilename="../../../log/sbert-crossencocer-test")

In [ ]:
from transformers import BertModel, BertTokenizer, DistilBertModel, DistilBertTokenizer, AlbertModel, AlbertTokenizer

# CrossEncoder 모델 로딩
# => NLI 학습된 bert 모델인 경우에는 bertmodel로 만들어야 함(왜냐하면 NLI는 classifiation 모델이므로, 출력값이 3개로 됨)
#model_path = "../../../model/bert/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0327"
 
nlimodel = False
#model_path = "../../../model/classification/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0327-ft-nli-0328/bertmodel"
#model_path = "../../../model/distilbert/distilbert-0331-TS-nli-0.1-10/bertmodel"
model_path = "../../../data11/model/moco/cross/albert-small-kor-cross-nli"

if nlimodel:
    # NLI 학습된 bert 모델인 경우에는 bertmodel로 만들어야 함
    # **왜냐하면 NLI는 classifiation 모델이므로, 출력값(참, 거짓, 중립)이 3개이므로, 출력은 임베딩값 출력되는 모델로 변환시켜야함
    #tokenizer = BertTokenizer.from_pretrained(model_path, do_lower_case=False)
    #bertmodel = BertModel.from_pretrained(model_path)
    tokenizer = AlbertTokenizer.from_pretrained(model_path, do_lower_case=True, keep_accent=False)
    bertmodel = AlbertModel.from_pretrained(model_path)
    OUTPATH = model_path + "/bertmodel"
    os.makedirs(OUTPATH, exist_ok=True)
    bertmodel.save_pretrained(OUTPATH)
    tokenizer.save_pretrained(OUTPATH)
    
    model = CrossEncoder(OUTPATH)        # CrossEncoder NLI -> BERT 모델로 변환된 모델 로딩
else:    
    model = CrossEncoder(model_path)     # CrossEncoder 모델 로딩

In [ ]:
corpus = ['서울은 대한민국에 수도이며, 정치 경제 중심지이다',
          '내년 경제 성장은 4%대 성장을 이룰거라 예상된다',
          '프랑스 파리는 전세계 관광객들이 매년 찾는 관광도시이다',
          '올해에는 대통령 선거와 지방선거가 동시에 열린다',
          '오늘 날씨는 비가 내리고 매우 춥다',
          '손홍민이 영국 프리미어 축구 경기에서 11번째 골을 넣었다',
          '건조한 날씨에 산불을 조심해야 한다',
          '윈도우11 OS에 검색 기능을 강화 하였다',
          '한국은행은 올해 하반기 금리를 동결했다',
          'Going on a trip',
          'it is raining',
          'stock market opened',
          'There is a voting for the class president election',
          '오늘은 흐리고, 바람이 세다',
          '대한민국',
          '대한민국의 보수정당',
          '대한민국 육군사관학교',
          '대한민국의_소방',
          '제주도',
          '서울',
         ]
          
query = '대한민국'

In [ ]:
# query와 corpus들을 묶어서 입력 문장 만듬
sentence_combinations = [[query, corpus_sentence] for corpus_sentence in corpus]
print(sentence_combinations)

In [ ]:
# 예측하여 스코어들을 구함
logger.info(f"============================================================")
logger.info(f'model : {model_path}')
start = time.time()

scores = model.predict(sentence_combinations)

logger.info(f'*처리시간:{time.time()-start}\n')

# 내림 차순으로 정렬
dec_scores = reversed(np.argsort(scores))

# 출력
logger.info(f"query : {query}")
for idx in dec_scores:
    print(idx)
    logger.info("{:.2f}\t{}".format(scores[idx], corpus[idx]))
logger.info(f"============================================================")